In [ ]:
import numpy as np
import pandas as pd

def get_agent_path_table(agent_data, new_od_dis):
    # 保留所需的代理信息列
    agent_data = agent_data[['trip_id', 'ids', 'O', 'D', 'date', 'start_step']].copy()

    # 合并数据
    merge_data = pd.merge(agent_data, new_od_dis, left_on=['O', 'D'], right_on=['station_id_x', 'station_id_y'])

    # 预计算每个组的累积概率
    merge_data['cumulative_probability'] = merge_data.groupby(['trip_id', 'ids', 'O', 'D'])['probability'].cumsum()

    # 定义向量化的抽样函数
    def vectorized_sample(group):
        if group.empty:
            return None  # 如果组是空的，返回None或适当的默认值
        rand_num = np.random.rand()
        chosen_idx = group['cumulative_probability'].searchsorted(rand_num, side='right')
        if chosen_idx >= len(group):
            return None  # 防止chosen_idx超出数组边界
        return group.iloc[chosen_idx]

    # 应用向量化抽样
    sampled_paths = merge_data.groupby(['trip_id', 'ids', 'O', 'D']).apply(vectorized_sample).reset_index(drop=True)

    # 过滤掉因为空组或其他原因返回None的行
    sampled_paths = sampled_paths.dropna()

    # 选择需要的列并重命名
    result_columns = ['trip_id', 'ids', 'O', 'D', 'date', 'start_step', 'edge_id', 'cumsumlength']
    final_result = sampled_paths[result_columns].rename(columns={'edge_id': 'path'})

    return final_result
